In [1]:
import numpy as np
from PIL import Image
from typing import List, Optional
Image.MAX_IMAGE_PIXELS=None

In [2]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [3]:
from eval.interpolator import Interpolator
it = Interpolator(r'\\shelter\Kyu\3D_skin\pretrained_models\film_net\Style\saved_model')

In [4]:
x0=np.array(Image.open(r"\\fatherserverdw\Saurabh\Saurabh\Pancreas_Ashley_Files\registered_10x_images\TC_001_0001.tif"))
x1=np.array(Image.open(r"\\fatherserverdw\Saurabh\Saurabh\Pancreas_Ashley_Files\registered_10x_images\TC_001_0003.tif"))

In [6]:
x1.shape

(27968, 29712, 3)

In [79]:
def image_to_patches(image: np.ndarray, block_shape: List[int]) -> np.ndarray:
  """Folds an image into patches and stacks along the batch dimension.

  Args:
    image: The input image of shape [B, H, W, C].
    block_shape: The number of patches along the height and width to extract.
      Each patch is shaped (H/block_shape[0], W/block_shape[1])

  Returns:
    The extracted patches shaped [num_blocks, patch_height, patch_width,...],
      with num_blocks = block_shape[0] * block_shape[1].
  """
  block_height, block_width = block_shape
  num_blocks = block_height * block_width

  height, width, channel = image.shape[-3:]
  patch_height, patch_width = height//block_height, width//block_width

  assert height == (
      patch_height * block_height
  ), 'block_height=%d should evenly divide height=%d.'%(block_height, height)
  assert width == (
      patch_width * block_width
  ), 'block_width=%d should evenly divide width=%d.'%(block_width, width)

  patch_size = patch_height * patch_width
  paddings = 2*[[0, 0]]

  patches = tf.space_to_batch(image, [patch_height, patch_width], paddings)
  patches = tf.split(patches, patch_size, 0)
  patches = tf.stack(patches, axis=3)
  patches = tf.reshape(patches,
                       [num_blocks, patch_height, patch_width, channel])
  return patches.numpy()


def patches_to_image(patches: np.ndarray, block_shape: List[int]) -> np.ndarray:
  """Unfolds patches (stacked along batch) into an image.

  Args:
    patches: The input patches, shaped [num_patches, patch_H, patch_W, C].
    block_shape: The number of patches along the height and width to unfold.
      Each patch assumed to be shaped (H/block_shape[0], W/block_shape[1]).

  Returns:
    The unfolded image shaped [B, H, W, C].
  """
  block_height, block_width = block_shape
  # paddings = 2 * [[0, 0, 0]]

  patch_height, patch_width, channel = patches.shape[-3:]
  patch_size = patch_height * patch_width

  patches = tf.reshape(patches,
                       [1, block_height, block_width, patch_size, channel])
  patches = tf.split(patches, patch_size, axis=3)
  patches = tf.stack(patches, axis=0)
  patches = tf.reshape(patches,
                       [patch_size, block_height, block_width, channel])
  image = tf.batch_to_space(patches, [patch_height, patch_width], paddings)
  return image.numpy()


In [77]:
block_shape=( 1024,1024)

In [64]:
image = x0
block_height, block_width = block_shape
num_blocks = block_height * block_width

In [65]:
image = x0[0:1024*2, 0:1024*2]

In [66]:
image = x0[0:(27968//1024)*1024, 0:(27968//1024)*1024]
image.shape

(27648, 27648, 3)

In [60]:
x0[0:(27968//1024)*1024, 0:(27968//1024)*1024].shape

(27648, 27648, 3)

In [52]:
(27968//1024)*1024

SyntaxError: invalid syntax (1301612043.py, line 1)

In [99]:
image=x0[0:(27968//1024)*1024,0:(27968//1024)*1024]
block_shape=(1024,1024)
block_height, block_width = block_shape
num_blocks = block_height * block_width
height, width, channel = image.shape[-3:]
patch_height, patch_width = height//block_height, width//block_width

assert height == (
    patch_height * block_height
), 'block_height=%d should evenly divide height=%d.'%(block_height, height)
assert width == (
    patch_width * block_width
), 'block_width=%d should evenly divide width=%d.'%(block_width, width)

patch_size = patch_height * patch_width
paddings = 2*[[0, 0]]

patches = tf.space_to_batch_nd(image, [patch_height, patch_width], paddings)


InvalidArgumentError: padded_shape[1]=3 is not divisible by block_shape[1]=27 [Op:SpaceToBatchND]

29

In [97]:

patch_height, patch_width = height//block_height, width//block_width

assert height == (
    patch_height * block_height
), 'block_height=%d should evenly divide height=%d.'%(block_height, height)
assert width == (
    patch_width * block_width
), 'block_width=%d should evenly divide width=%d.'%(block_width, width)

patch_size = patch_height * patch_width
paddings = 2*[[0, 0]]

patches = tf.space_to_batch_nd(image, [patch_height, patch_width], paddings)

AssertionError: block_height=1024 should evenly divide height=27.

In [100]:
x0_patches = image_to_patches(x0[0:(24576//24), 0:24576], block_shape)

InvalidArgumentError: padded_shape[1]=3 is not divisible by block_shape[1]=24 [Op:SpaceToBatchND]

In [85]:
x0_patches = image_to_patches(x0[0:(27968//3*1024)*3*1024, 0:(27968//3*1024)*3*1024], block_shape)
#x1_patches = image_to_patches(x1, block_shape)

AssertionError: block_height=1024 should evenly divide height=27968.

In [ ]:
x0_patches = image_to_patches(x0, block_shape)
x1_patches = image_to_patches(x1, block_shape)
dt = 0.5
# Run the interpolator on each patch pair.
output_patches = []
for image_0, image_1 in zip(x0_patches, x1_patches):
    mid_patch = it.interpolate(image_0[np.newaxis, ...],
                                 image_1[np.newaxis, ...], dt)
    output_patches.append(mid_patch)

# Reconstruct interpolated image by stitching interpolated patches.
output_patches = np.concatenate(output_patches, axis=0)
output_im = patches_to_image(output_patches, self._block_shape)